In [1]:
import pandas as pd
ogdata=pd.read_csv('e_commerce.csv')
print(ogdata.describe(), '\n',ogdata.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB
            Quantity      UnitPrice     CustomerID
count  541909.000000  541909.000000  406829.000000
mean        9.552250       4.611114   15287.690570
std       218.081158      96.759853    1713.600303
min    -80995.000000  -11062.060000   12346.000000
25%         1.000000       1.250000   13953.000000
50%         3.000000       2.080000   15152.000000
75%        10.000000       4.130000   16

共计541909观察值，description与CustomerID存在缺失值
InvoiceDate应为日期，CustomerID应为字符串

In [2]:
cleaned_data = ogdata.copy()
cleaned_data["InvoiceDate"] = pd.to_datetime(cleaned_data["InvoiceDate"])
cleaned_data["CustomerID"] = cleaned_data["CustomerID"].astype(str)
cleaned_data["CustomerID"] = cleaned_data["CustomerID"].str.slice(0, -2)
print(cleaned_data["InvoiceDate"],'\n',cleaned_data["CustomerID"])

0        2010-12-01 08:26:00
1        2010-12-01 08:26:00
2        2010-12-01 08:26:00
3        2010-12-01 08:26:00
4        2010-12-01 08:26:00
                 ...        
541904   2011-12-09 12:50:00
541905   2011-12-09 12:50:00
541906   2011-12-09 12:50:00
541907   2011-12-09 12:50:00
541908   2011-12-09 12:50:00
Name: InvoiceDate, Length: 541909, dtype: datetime64[ns] 
 0         17850
1         17850
2         17850
3         17850
4         17850
          ...  
541904    12680
541905    12680
541906    12680
541907    12680
541908    12680
Name: CustomerID, Length: 541909, dtype: object


In [3]:
print(ogdata[ogdata['Quantity']<0])

       InvoiceNo StockCode                       Description  Quantity  \
141      C536379         D                          Discount        -1   
154      C536383    35004C   SET OF 3 COLOURED  FLYING DUCKS        -1   
235      C536391     22556    PLASTERS IN TIN CIRCUS PARADE        -12   
236      C536391     21984  PACK OF 12 PINK PAISLEY TISSUES        -24   
237      C536391     21983  PACK OF 12 BLUE PAISLEY TISSUES        -24   
...          ...       ...                               ...       ...   
540449   C581490     23144   ZINC T-LIGHT HOLDER STARS SMALL       -11   
541541   C581499         M                            Manual        -1   
541715   C581568     21258        VICTORIAN SEWING BOX LARGE        -5   
541716   C581569     84978  HANGING HEART JAR T-LIGHT HOLDER        -1   
541717   C581569     20979     36 PENCILS TUBE RED RETROSPOT        -5   

            InvoiceDate  UnitPrice  CustomerID         Country  
141      12/1/2010 9:41      27.50     14527.0

quantity为负的变量，InvoiceNO均以C开头,需要检验是否开头不为C的值

In [4]:
print(ogdata[(ogdata["Quantity"] < 0) & (ogdata["InvoiceNo"].str[0] != "C")])

       InvoiceNo StockCode Description  Quantity      InvoiceDate  UnitPrice  \
2406      536589     21777         NaN       -10  12/1/2010 16:50        0.0   
4347      536764    84952C         NaN       -38  12/2/2010 14:42        0.0   
7188      536996     22712         NaN       -20  12/3/2010 15:30        0.0   
7189      536997     22028         NaN       -20  12/3/2010 15:30        0.0   
7190      536998     85067         NaN        -6  12/3/2010 15:30        0.0   
...          ...       ...         ...       ...              ...        ...   
535333    581210     23395       check       -26  12/7/2011 18:36        0.0   
535335    581212     22578        lost     -1050  12/7/2011 18:38        0.0   
535336    581213     22576       check       -30  12/7/2011 18:38        0.0   
536908    581226     23090     missing      -338   12/8/2011 9:56        0.0   
538919    581422     23169     smashed      -235  12/8/2011 15:24        0.0   

        CustomerID         Country  
24

依然存在开头不为C的观察值

In [5]:
print(ogdata[(ogdata["Quantity"] < 0)&(ogdata["InvoiceNo"].str[0] != "C")&(ogdata["UnitPrice"] != 0)])

Empty DataFrame
Columns: [InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country]
Index: []


根据输出结果，猜想得到验证，当`Quantity`为负数的时候，观察值满足以下条件之一：
- `InvoiceNo`以C开头，表示订单被取消
- `UnitPrice`为0，说明单价为0英镑

这些交易数据均不是有效成交数据，不贡献销售，不在后续分析范围内，因此我们将在数据清理步骤，将`Quantity`为负数的观察值删除。

In [6]:
cleaned_data = cleaned_data[cleaned_data["Quantity"] >= 0]
cleaned_data = cleaned_data[cleaned_data["UnitPrice"] >= 0]

In [7]:
A=ogdata['Description'].isnull()
print(ogdata[A])

       InvoiceNo StockCode Description  Quantity      InvoiceDate  UnitPrice  \
622       536414     22139         NaN        56  12/1/2010 11:52        0.0   
1970      536545     21134         NaN         1  12/1/2010 14:32        0.0   
1971      536546     22145         NaN         1  12/1/2010 14:33        0.0   
1972      536547     37509         NaN         1  12/1/2010 14:33        0.0   
1987      536549    85226A         NaN         1  12/1/2010 14:34        0.0   
...          ...       ...         ...       ...              ...        ...   
535322    581199     84581         NaN        -2  12/7/2011 18:26        0.0   
535326    581203     23406         NaN        15  12/7/2011 18:31        0.0   
535332    581209     21620         NaN         6  12/7/2011 18:35        0.0   
536981    581234     72817         NaN        27  12/8/2011 10:33        0.0   
538554    581408     85175         NaN        20  12/8/2011 14:06        0.0   

        CustomerID         Country  
62

依照输出结果，缺失description的数据UnitPrice均为0，需要检查其他缺失description的unitprice是否也为0

In [8]:
print(ogdata[A & ogdata['UnitPrice']!=0])

Empty DataFrame
Columns: [InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country]
Index: []


缺失description的，unitprice确实均为0，需要检查是否有缺失description但unit price不为0

In [9]:
cleaned_data.dropna(subset=["Description"], inplace=True)

In [10]:
print(ogdata[(ogdata["Description"].isnull()) & (ogdata["UnitPrice"] != 0)])

Empty DataFrame
Columns: [InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country]
Index: []


筛选出来结果数量为0条，说明缺失description值的数据，同时也不具备有效的UnitPrice值，可以删除

In [11]:
print(ogdata[ogdata["CustomerID"].isnull()])

       InvoiceNo StockCode                      Description  Quantity  \
622       536414     22139                              NaN        56   
1443      536544     21773  DECORATIVE ROSE BATHROOM BOTTLE         1   
1444      536544     21774  DECORATIVE CATS BATHROOM BOTTLE         2   
1445      536544     21786               POLKADOT RAIN HAT          4   
1446      536544     21787            RAIN PONCHO RETROSPOT         2   
...          ...       ...                              ...       ...   
541536    581498    85099B          JUMBO BAG RED RETROSPOT         5   
541537    581498    85099C   JUMBO  BAG BAROQUE BLACK WHITE         4   
541538    581498     85150    LADIES & GENTLEMEN METAL SIGN         1   
541539    581498     85174                S/4 CACTI CANDLES         1   
541540    581498       DOT                   DOTCOM POSTAGE         1   

            InvoiceDate  UnitPrice  CustomerID         Country  
622     12/1/2010 11:52       0.00         NaN  United Kin

无影响

In [12]:
print(ogdata['Country'].value_counts())

United Kingdom          495266
Germany                   9495
France                    8557
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
China                      288
Singapore                  229
USA                        218
UK                         211
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United States               73
United Arab Emirates        68
European

国家名称存在重复

In [13]:
cleaned_data["Country"] = cleaned_data["Country"].replace({"UK": "United Kingdom", "U.K.": "United Kingdom"})
cleaned_data["Country"] = cleaned_data["Country"].replace({ "USA":"United States"})
print(cleaned_data['Country'].value_counts())

United Kingdom          485692
Germany                   9042
France                    8408
EIRE                      7894
Spain                     2485
Netherlands               2363
Belgium                   2031
Switzerland               1967
Portugal                  1501
Australia                 1185
Norway                    1072
Italy                      758
Channel Islands            748
Finland                    685
Cyprus                     614
Sweden                     451
Unspecified                446
Austria                    398
Denmark                    380
Poland                     330
Japan                      321
Israel                     295
China                      284
Singapore                  222
Iceland                    182
United States              179
Canada                     151
Greece                     145
Malta                      112
United Arab Emirates        68
European Community          60
RSA                         58
Lebanon 

In [14]:
cleaned_data.sample(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
125882,547063,21791,VINTAGE HEADS AND TAILS CARD GAME,2,2011-03-20 13:33:00,1.25,14769,United Kingdom
284534,561876,82600,NO SINGING METAL SIGN,2,2011-07-31 12:08:00,2.10,14472,United Kingdom
122310,546861,22228,BUNNY WOODEN PAINTED WITH BIRD,12,2011-03-17 15:03:00,0.85,14911,EIRE
106846,545346,22283,6 EGG HOUSE PAINTED WOOD,2,2011-03-02 09:49:00,7.95,12834,United Kingdom
40478,539737,72349b,SET/6 PURPLE BUTTERFLY T-LIGHTS,2,2010-12-21 15:19:00,4.21,n,United Kingdom
142879,548650,22832,BROCANTE SHELF WITH HOOKS,2,2011-04-01 13:22:00,10.75,14852,United Kingdom
235850,557645,23181,BULL DOG BOTTLE TOP WALL CLOCK,1,2011-06-21 17:08:00,16.63,n,United Kingdom
540266,581474,22906,12 MESSAGE CARDS WITH ENVELOPES,1,2011-12-08 20:01:00,1.65,12748,United Kingdom
241106,558183,22679,FRENCH BLUE METAL DOOR SIGN 4,2,2011-06-27 12:13:00,1.25,15996,United Kingdom
534237,581141,22813,PACK 3 BOXES BIRD PANETTONE,12,2011-12-07 13:35:00,1.95,14132,United Kingdom


In [15]:
cleaned_data.to_csv("e_commerce_cleaned.csv", index=False)
pd.read_csv("e_commerce_cleaned.csv").sample(10)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_3652\2553602636.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv("e_commerce_cleaned.csv").sample(10)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
486409,578347,23229,VINTAGE DONKEY TAIL GAME,1,2011-11-24 09:26:00,7.46,n,United Kingdom
287370,562680,22570,FELTCRAFT CUSHION RABBIT,1,2011-08-08 13:46:00,3.75,17954,United Kingdom
256447,559910,23188,VINTAGE 2 METER FOLDING RULER,12,2011-07-13 14:29:00,1.65,18245,United Kingdom
22795,538188,22317,FIVE CATS HANGING DECORATION,1,2010-12-10 10:29:00,2.95,n,United Kingdom
102323,545186,22375,AIRLINE BAG VINTAGE JET SET BROWN,1,2011-02-28 15:05:00,4.25,17841,United Kingdom
344461,567671,22551,PLASTERS IN TIN SPACEBOY,1,2011-09-21 15:34:00,3.29,n,United Kingdom
161875,550819,22301,COFFEE MUG CAT + BIRD DESIGN,6,2011-04-20 17:37:00,2.55,17530,United Kingdom
328841,566314,22732,3D VINTAGE CHRISTMAS STICKERS,18,2011-09-12 09:57:00,1.25,15358,United Kingdom
33756,539391,22770,MIRROR CORNICE,2,2010-12-17 11:51:00,14.95,12417,Belgium
415267,573286,23275,SET OF 3 HANGING OWLS OLLIE BEAK,24,2011-10-28 14:38:00,1.25,n,Switzerland
